In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models

#加載MNIST數據

In [ ]:
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist["data"], mnist["target"].astype(int)

# 預處理數據

In [ ]:
X = X / 255.0
X = X.values.reshape(-1, 28, 28, 1)
y = y.values

# 分割數據集

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 定義ResNet模型

In [ ]:
def resnet_block(inputs, filters, kernel_size=3, stride=1, activation='relu'):
    x = layers.Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    if activation:
        x = layers.Activation(activation)(x)
    return x

def resnet_v1(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    x = resnet_block(inputs, 64)
    x = resnet_block(x, 64)
    x = layers.MaxPooling2D(pool_size=2)(x)

    x = resnet_block(x, 128, stride=2)
    x = resnet_block(x, 128)
    x = layers.MaxPooling2D(pool_size=2)(x)

    x = resnet_block(x, 256, stride=2)
    x = resnet_block(x, 256)
    x = layers.MaxPooling2D(pool_size=2)(x)

    x = resnet_block(x, 512, stride=2)
    x = resnet_block(x, 512)
    x = layers.GlobalAveragePooling2D()(x)

    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    return model

# 創建ResNet模型

In [ ]:
model = resnet_v1(input_shape=(28, 28, 1), num_classes=10)

# 編譯模型

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 訓練模型

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# 評估模型

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'Accuracy: {test_acc * 100}%')